# EDA - HDB Resale Price for ELT

In [16]:
import json
import requests
import time
import pandas as pd

## Setup
1. Paste the dataset ID you copied into the cell below
2. Run All Cells (click `Runtime` -> `Run All`)

In [17]:
# Dataset ID for the EDA notebook
DATASET_ID = "d_8b84c4ee58e3cfc0ece0d773c8ca6abc" # e.g. "d_69b3380ad7e51aff3a7dcc84eba52b8a"

## Dataset and Column Metadata

In [18]:
# code supplied by data.gov.sg to download the dataset metadata
s = requests.Session()
s.headers.update({'referer': 'https://colab.research.google.com'})
base_url = "https://api-production.data.gov.sg"
url = base_url + f"/v2/public/api/datasets/{DATASET_ID}/metadata"
#print(url)
response = s.get(url)
data = response.json()['data']
columnMetadata = data.pop('columnMetadata', None)

print("Dataset Metadata:")
print(json.dumps(data, indent=2))

print("\nColumns:\n", list(columnMetadata['map'].values()))


Dataset Metadata:
{
  "datasetId": "d_8b84c4ee58e3cfc0ece0d773c8ca6abc",
  "createdAt": "2021-07-28T10:37:00+08:00",
  "name": "Resale flat prices based on registration date from Jan-2017 onwards",
  "collectionIds": [
    "189"
  ],
  "description": "Notes:\n\n1. The approximate floor area includes any recess area purchased, space adding item under HDB\u2019s upgrading programmes, roof terrace, etc.\n\n2. The transactions exclude resale transactions that may not reflect the full market price such as resale between relatives and resale of part shares.\n\n3. Resale prices should be taken as indicative only as the resale prices agreed between buyers and sellers are dependent on many factors.",
  "format": "CSV",
  "lastUpdatedAt": "2025-07-11T02:09:19+08:00",
  "managedBy": "Housing & Development Board",
  "coverageStart": "2017-01-01T08:00:00+08:00",
  "coverageEnd": "2025-07-01T08:00:00+08:00",
  "contactEmails": [
    "lau_pei_wen@hdb.gov.sg"
  ],
  "datasetSize": 20840923
}

Columns:

## Download File

In [19]:
# code supplied by data.gov.sg to download the dataset
# Function to download the dataset

def download_file(DATASET_ID):
  # initiate download
  initiate_download_response = s.get(
      f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/initiate-download",
      headers={"Content-Type":"application/json"},
      json={}
  )
  #print(initiate_download_response.json()['data']['message'])

  # poll download
  MAX_POLLS = 5
  for i in range(MAX_POLLS):
    poll_download_response = s.get(
        f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/poll-download",
        headers={"Content-Type":"application/json"},
        json={}
    )
    #print("Poll download response:", poll_download_response.json())
    if "url" in poll_download_response.json()['data']:
      #print(poll_download_response.json()['data']['url'])
      DOWNLOAD_URL = poll_download_response.json()['data']['url']
      df = pd.read_csv(DOWNLOAD_URL)

      display(df.head())
      print("\nDataframe loaded!")
      return df
    if i == MAX_POLLS - 1:
      print(f"{i+1}/{MAX_POLLS}: No result found, possible error with dataset, please try again or let us know at https://go.gov.sg/datagov-supportform\n")
    else:
      print(f"{i+1}/{MAX_POLLS}: No result yet, continuing to poll\n")
    time.sleep(3)

df = download_file(DATASET_ID)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0



Dataframe loaded!


In [20]:
if df is not None:
	display(df.describe())
else:
	print("DataFrame 'df' is None. Please check if the dataset was loaded successfully.")

,floor_area_sqm,lease_commence_date,resale_price
count,211086.000000,211086.000000,2.110860e+05
mean,96.857005,1996.341666,5.175608e+05
std,24.031943,14.244057,1.827699e+05
min,31.000000,1966.000000,1.400000e+05
25%,82.000000,1985.000000,3.800000e+05
50%,93.000000,1996.000000,4.850000e+05
75%,112.000000,2011.000000,6.200000e+05
max,366.700000,2022.000000,1.658888e+06


In [21]:
if df is not None:
	df.to_csv("./data/hdb_resale_price.csv", index=False)
else:
	print("DataFrame 'df' is None. Please check if the dataset was downloaded successfully.")

## Load Data

In [22]:
# Load raw data from the CSV file
df = pd.read_csv("./data/hdb_resale_price.csv")

## Data Check

In [23]:
# Check for duplicates
duplicates = df[df.duplicated(keep=False)]
if not duplicates.empty:
    print("Duplicates found:")
    print(duplicates)
else:
    print("No duplicates found.")

Duplicates found:
          month          town flat_type block       street_name storey_range  \
224     2017-01   BUKIT MERAH    4 ROOM   106    HENDERSON CRES     07 TO 09   
243     2017-01   BUKIT MERAH    4 ROOM   106    HENDERSON CRES     07 TO 09   
304     2017-01  CENTRAL AREA    3 ROOM   271          QUEEN ST     16 TO 18   
305     2017-01  CENTRAL AREA    3 ROOM   271          QUEEN ST     16 TO 18   
505     2017-01   JURONG EAST    4 ROOM   265       TOH GUAN RD     04 TO 06   
...         ...           ...       ...   ...               ...          ...   
205748  2025-04     SEMBAWANG    4 ROOM  103B       CANBERRA ST     04 TO 06   
206804  2025-04      SENGKANG    4 ROOM  457B  SENGKANG WEST RD     13 TO 15   
206805  2025-04      SENGKANG    4 ROOM  457B  SENGKANG WEST RD     13 TO 15   
210480  2025-05        YISHUN    4 ROOM  426B     YISHUN AVE 11     10 TO 12   
210483  2025-05        YISHUN    4 ROOM  426B     YISHUN AVE 11     10 TO 12   

        floor_area_sq

In [24]:
# Show the duplicates for visual inspection
df[df.duplicated(keep=False)].sort_values(by=list(df.columns))  

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
224,2017-01,BUKIT MERAH,4 ROOM,106,HENDERSON CRES,07 TO 09,81.0,Improved,1975,57 years,470000.0
243,2017-01,BUKIT MERAH,4 ROOM,106,HENDERSON CRES,07 TO 09,81.0,Improved,1975,57 years,470000.0
304,2017-01,CENTRAL AREA,3 ROOM,271,QUEEN ST,16 TO 18,68.0,Improved,1979,61 years 02 months,470000.0
305,2017-01,CENTRAL AREA,3 ROOM,271,QUEEN ST,16 TO 18,68.0,Improved,1979,61 years 02 months,470000.0
505,2017-01,JURONG EAST,4 ROOM,265,TOH GUAN RD,04 TO 06,101.0,Model A,1998,80 years 09 months,470000.0
...,...,...,...,...,...,...,...,...,...,...,...
201755,2025-05,HOUGANG,4 ROOM,994C,BUANGKOK LINK,10 TO 12,93.0,Model A,2021,94 years 09 months,675000.0
210480,2025-05,YISHUN,4 ROOM,426B,YISHUN AVE 11,10 TO 12,92.0,Model A,2015,88 years 11 months,595000.0
210483,2025-05,YISHUN,4 ROOM,426B,YISHUN AVE 11,10 TO 12,92.0,Model A,2015,88 years 11 months,595000.0
205461,2025-06,SEMBAWANG,2 ROOM,103A,CANBERRA ST,04 TO 06,38.0,2-room,2020,94 years 07 months,350000.0


**There could be two possible reasons for duplications. The first reason is duplicate submission to the system. The second reason is there is a very slim possibility that there could be a similar sales at the same location, same type, within the same 3 storey range and sell at a same price.**

In [25]:
# compute the percentage of duplicates in the dataset
duplicates_percentage = df.duplicated().sum() / len(df) * 100
print(f'The number of duplicates is {df.duplicated().sum()}, which is {duplicates_percentage:.2f}% of the entire dataset. ')

The number of duplicates is 304, which is 0.14% of the entire dataset. 


**Since duplicated data is less than 1% of the entire dataset, we will remove the duplicates as it would not have impacted our analysis or prediction.**

In [26]:
# Remove duplicates
df = df.drop_duplicates()

In [27]:
# check if there are any duplicates in the dataset for confirmation
df.duplicated().sum()

np.int64(0)

In [28]:
len(df)

210782

### Detecting Missing Values

In [29]:
# check if there are any missing values in the dataset
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

### Data Structure Analysis

In [30]:
# display the information and data structure of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210782 entries, 0 to 211085
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                210782 non-null  object 
 1   town                 210782 non-null  object 
 2   flat_type            210782 non-null  object 
 3   block                210782 non-null  object 
 4   street_name          210782 non-null  object 
 5   storey_range         210782 non-null  object 
 6   floor_area_sqm       210782 non-null  float64
 7   flat_model           210782 non-null  object 
 8   lease_commence_date  210782 non-null  int64  
 9   remaining_lease      210782 non-null  object 
 10  resale_price         210782 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 19.3+ MB


## END